Installing Necessary libraries

In [1]:
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q gradio
!pip install -q pyChatGPT

     |████████████████████████████████| 5.8 MB 30.9 MB/s 
     |████████████████████████████████| 7.6 MB 61.5 MB/s 
     |████████████████████████████████| 182 kB 78.1 MB/s 
     |████████████████████████████████| 13.8 MB 196 kB/s 
     |████████████████████████████████| 84 kB 4.3 MB/s 
     |████████████████████████████████| 278 kB 71.6 MB/s 
     |████████████████████████████████| 56 kB 1.8 MB/s 
     |████████████████████████████████| 55 kB 2.6 MB/s 
     |████████████████████████████████| 106 kB 78.3 MB/s 
     |████████████████████████████████| 54 kB 3.4 MB/s 
     |████████████████████████████████| 2.3 MB 68.1 MB/s 
     |████████████████████████████████| 213 kB 80.2 MB/s 
     |████████████████████████████████| 84 kB 3.8 MB/s 
     |████████████████████████████████| 64 kB 3.3 MB/s 
     |████████████████████████████████| 80 kB 10.3 MB/s 
     |████████████████████████████████| 68 kB 8.5 MB/s 
     |████████████████████████████████| 68 kB 8.1 MB/s 
     |█████████████████████████

In [2]:
import whisper
import gradio as gr
import time
from pyChatGPT import pyChatGPT
import warnings

In [3]:
warnings.filterwarnings("ignore")

In [4]:
secret_token = "" # Enter your session token here!

In [5]:
model = whisper.load_model("base")

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 134MiB/s]


In [6]:
model.device

device(type='cuda', index=0)

Transcribe **Function**


In [10]:
def transcribe(audio):
  #load audio and pad/trim it to fit 30 seconds
  audio = whisper.load_audio(audio)
  audio = whisper.pad_or_trim(audio)

  #make log-mel spectrogram and move ro the same device as the model
  mol = whisper.log_mel_spectrogram(audio).to(model.device)

  #detect the spoken language
  _, probs = model.detect_language(mel)

  #decode the audio
  options = whisper.DecodingOptions()
  result = whisper.decode(model,mel,options)
  result_text = result.text

  #Pass the generated text to audio
  chatgpt_api = ChatGPT(secret_token)
  resp = chatgpt_api.send_message(result_text)
  out_result = resp['message']

  return [result_text, out_result]



Gradio Interface

In [11]:
output_1 = gr.Textbox(label="Speech to Text")
output_2 = gr.Textbox(label="ChatGPT Output")

gr.Interface(
    title = 'OpenAI Whisper and ChatGPT ASR Gradio Web UI',
    fn=transcribe,
    inputs=[
        gr.inputs.Audio(source="microphone" , type="filepath")
    ],

    outputs=[
        output_1, output_2
    ],
    live=True).launch()




Hint: Set streaming=True for Audio component to use live streaming.
Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>